# Risk Model Pipeline Demo

This notebook demonstrates how to use the risk-model-pipeline package.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from risk_pipeline import RiskModelPipeline
from risk_pipeline.core.config import Config

# Set display options
pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')

## 1. Load Sample Data

In [ ]:
# Load your data
# df = pd.read_csv('../data/sample_data.csv')

# For demo, create synthetic data
np.random.seed(42)
n_samples = 10000

df = pd.DataFrame({
    'app_id': range(n_samples),
    'app_dt': pd.date_range('2023-01-01', periods=n_samples, freq='H'),
    'feature1': np.random.randn(n_samples),
    'feature2': np.random.randn(n_samples) * 2 + 1,
    'feature3': np.random.choice(['A', 'B', 'C'], n_samples),
    'feature4': np.random.exponential(2, n_samples),
    'target': np.random.binomial(1, 0.3, n_samples)
})

print(f"Data shape: {df.shape}")
print(f"Target distribution:\n{df['target'].value_counts()}")
df.head()

## 2. Configure Pipeline

In [ ]:
# Configure the pipeline
config = Config(
    target_col='target',
    id_col='app_id',
    time_col='app_dt',
    output_folder='../output/demo',
    
    # Feature selection parameters
    iv_min=0.02,
    psi_threshold=0.25,
    
    # Model parameters
    model_type='lightgbm',
    use_optuna=False,
    
    # Split parameters
    oot_months=1,
    
    # Report parameters
    output_excel_path='model_report.xlsx'
)

print("Configuration set!")

## 3. Run Pipeline

In [ ]:
# Initialize pipeline
pipeline = RiskModelPipeline(config)

# Run the pipeline
results = pipeline.run(df)

print("\n" + "="*50)
print("Pipeline Results:")
print("="*50)
print(f"Best Model: {pipeline.best_model_name_}")
print(f"Train AUC: {pipeline.best_auc_:.4f}")
print(f"Number of features selected: {len(pipeline.final_vars_)}")

## 4. Analyze Results

In [ ]:
# Feature importance
if hasattr(pipeline.best_model_, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'feature': pipeline.final_vars_,
        'importance': pipeline.best_model_.feature_importances_
    }).sort_values('importance', ascending=False)
    
    plt.figure(figsize=(10, 6))
    plt.barh(feature_importance['feature'][:10], feature_importance['importance'][:10])
    plt.xlabel('Importance')
    plt.title('Top 10 Feature Importance')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

## 5. Score New Data

In [ ]:
# Create new data for scoring
new_data = pd.DataFrame({
    'app_id': range(100),
    'feature1': np.random.randn(100),
    'feature2': np.random.randn(100) * 2 + 1,
    'feature3': np.random.choice(['A', 'B', 'C'], 100),
    'feature4': np.random.exponential(2, 100)
})

# Score using the pipeline
# scores = pipeline.score(new_data)

print("Scoring complete!")